In [14]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Step: Load data

In [45]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import numpy as np
import pandas as pd
import gower
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)


path_data = here("./data")
os.chdir(path_data)
data_draft_kings = pd.read_parquet("data_draft_kings_analysis.parquet")
data_draft_kings_analysis = data_draft_kings.drop(columns=["week"])

In [46]:
# Compute Gower distance matrix
distance_matrix = gower.gower_matrix(data_draft_kings_analysis)

In [47]:
data_clusters = [2, 3, 4, 5, 6]

In [48]:
labels_out = []

for x in range(len(data_clusters)):
    # Create the hierarchical agglomerative clustering model
    clustering_in = AgglomerativeClustering(
        n_clusters=data_clusters[x], affinity="precomputed", linkage="complete"
    )

    clustering_in.fit(distance_matrix)
    labels = clustering_in.labels_

    labels_out.append(labels)

In [49]:
accuracy_out = []
for x in range(len(labels_out)):
    accuracy_in = silhouette_score(data_draft_kings_analysis, labels_out[x])
    accuracy_out.append(accuracy_in)

In [50]:
accuracy_out

[0.288035499192513,
 0.04312217895576275,
 -0.030924717001696758,
 -0.08095817523341031,
 -0.18009020473223336]

Step: Save 

In [51]:
labels = pd.DataFrame(labels_out[0]).rename(columns={0: "labels"})

In [52]:
data_draft_kings_analysis_labels = pd.concat(
    [data_draft_kings_analysis, labels], axis=1
)

In [53]:
data_draft_kings_analysis_labels.columns

Index(['points', 'salary', 'reduction', 'value', 'ownership_%', 'WR', 'RB',
       'QB', 'TE', 'FLEX',
       ...
       'buccaneers', 'henry_ruggs', 'ricky_seals_jones', 'noah_fant',
       'donovan_jones', 'steelers', '1', '2', '3', 'labels'],
      dtype='object', length=220)

In [55]:
data_draft_kings_cluster = (
    data_draft_kings_analysis_labels.groupby("labels").mean().reset_index().round(2)
)

data_draft_kings_cluster.to_csv("data_draft_kings_cluster.csv")